<a href="https://colab.research.google.com/github/MatthewPlaudis/Stock_Prediction_Project/blob/main/Sentiment_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#These functions were implemented for the layout of wsj file
#modifications will be needed if other layout formats are used

#Either modify the code (prefered method) such that the files are uploaded the way MA demonstrated or
#manually upload wsj, and the dictionary MA linked on discord, to colab
#rename the dictionary to 'master.csv'
#rename s&p500_with_labels to 's&p.csv'

#Pay no attention to my verbose 'brute force' implementation in places, I try to avoid python as much as 
#possible and therefore crafty pythonic list manipulations often elude me


!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import csv
import numpy as np


In [ ]:
#new word list for modified vader
new_words = 
{'rise': 10,'rising': 10,'rises': 10,'fall': -10,'falls': -10,'beats': 5,
 'crashing': -200,'crashes': -200, 'crashed': -200,'sores': 100,'breaks': 50,
 'surge': 100, 'surges': 100,'losses': -100,'drops': -50,'bear': -20, 
 'bull': 20, 'decline': -30,'recover': 30,'recovering': 30,'raises': 10,
 'tougher': -10,'beating': -50,'down': -30,'below': -30,'sinks': -30,
}

In [ ]:
#word dictionary functions

def getDict():
  #load word Dictionary
  with open('master.csv', newline='') as f:
     reader = csv.reader(f)
     wordDict = list(reader)

#clean wordDict
  wordDictCleaned = {}
  for i in range (1,len(wordDict)):
   wordDictCleaned[wordDict[i][0]] = int(wordDict[i][8])-int(wordDict[i][7])

  return wordDictCleaned

In [ ]:
#wsj functions 

#reads and cleans wsj
def getWSJ():
  #input wsj
  with open('wsj.csv', newline='') as f:
     reader = csv.reader(f)
     wsj = list(reader)

  #clean wsj
  wsjCleanedtemp = []
  for i in range(1,len(wsj)):
   wsjCleanedtemp.append(wsj[i][1])

  #remove ';'
  wsjCleaned = []
  for i in range(0,len(wsjCleanedtemp)):
   hold = wsjCleanedtemp[i].replace(';',' ')
   wsjCleaned.append(hold)

  return wsjCleaned




# compile wsj news headings and its sentiment
def joinSent(wsjSentiment,wsjCleaned):
  combined = [['wsj News','sentiment']]
  for i in range (0,len(wsjSentiment)):
   hold = []
   hold.append(wsjCleaned[i])
   hold.append(wsjSentiment[i])
   combined.append(hold)
  return combined



#calculates sentiment
#chooser ==1: modified, chooser==2: wordDict, else OOB
def calc_wsj_sent(chooser,new_words,wordDictCleaned,wsjCleaned):
  wsjSentiment = []
  analyzer = SentimentIntensityAnalyzer()
  name = 'OOB'

  #modified
  if (chooser == 1):
    analyzer.lexicon.update(new_words)
    name = 'modified'
  
  #Worddict
  if (chooser==2):
    analyzer.lexicon.update(wordDictCleaned)
    name = 'wordDict'

  for i in range (0,len(wsjCleaned)):
    temp = analyzer.polarity_scores(wsjCleaned[i])
    wsjSentiment.append(temp.get('pos')-temp.get('neg'))
  
  #combine wsj and its sentiment
  joined = joinSent(wsjSentiment, wsjCleaned)

  #save wsj to file
  file = open('wsj_with_'+name+'_sentiment.csv','w+',newline = '')
  with file:
    write = csv.writer(file)
    write.writerows(joined)




In [ ]:
#functions for calculating wsj with OOB sent and s&p

def getFiles():
  #input s&p
  with open('s&p.csv', newline='') as f:
     reader = csv.reader(f)
     sAndP = list(reader)


  #input wsjOOB
  with open('wsj_with_OOB_sentiment.csv', newline='') as f:
     reader = csv.reader(f)
     wsjOOB = list(reader)


  #input wsj
  with open('wsj.csv', newline='') as f:
     reader = csv.reader(f)
     wsj = list(reader)

  return sAndP,wsjOOB,wsj


#join wsj and wsjOOB together
def joinWSJ(wsj,wsjOOB):
  grouped = []
  for i in range (1,len(wsj)):
    temp = []
    temp.append(wsj[i][0])
    temp.append(wsjOOB[i][0])
    temp.append(wsjOOB[i][1])
    grouped.append(temp)

  return grouped




#calculate averages per day
def calcAvg(grouped):
  hold = grouped[0][0]
  sumUp = float(grouped[0][2])
  count = 1
  groupedAvg = []
  for i in range(1,len(grouped)):
    if grouped[i][0] == hold:
     #keep averaging
      sumUp = sumUp + float(grouped[i][2])
      count = count + 1
    else:
      #save avg
      temp = []
      temp.append(hold)
      temp.append(sumUp/count)
      groupedAvg.append(temp)
      #reset
      hold = grouped[i][0]
      count = 1
      sumUp = float(grouped[i][2])
  return groupedAvg



#clean sAndP
def clean(sAndP):
  sAndPCleaned = []
  for i in range (2,len(sAndP)):
    sAndPCleaned.append(sAndP[i])
  return sAndPCleaned



#join by date
def join_up(sAndPCleaned,groupedAvg):
  sameDate = []
  hold= []
  hold.append('Date')
  hold.append('Open')
  hold.append('Adj Close')
  hold.append('Volume')
  hold.append("avg'd sentiment")
  hold.append('label')
  sameDate.append(hold)
  for i in range(0,len(sAndPCleaned)):
    for j in range(0,len(groupedAvg)):
      if sAndPCleaned[i][1] == groupedAvg[j][0]:
        temp = []
        temp.append(sAndPCleaned[i][1])
        temp.append(sAndPCleaned[i][2])
        temp.append(sAndPCleaned[i][6])
        temp.append(sAndPCleaned[i][7])
        temp.append(groupedAvg[j][1])
        temp.append(sAndPCleaned[i][8])
        sameDate.append(temp)
  
  return sameDate



#save wsj sentiment combined with s&p to file
def saveFile(sameDate):
  file = open('wsj_OOBsent_s&p.csv','w+',newline = '')
  with file:
    write = csv.writer(file)
    write.writerows(sameDate)


In [ ]:
#MAIN CODE STARTS HERE

In [ ]:
#calc raw sentiment files

#get cleaned word dict
wordDictCleaned = getDict()
#get cleaned wsj
wsjCleaned = getWSJ()


#sent_type == 1: modified, sent_type == 2: wordDict, else OOB
sent_type = 0;


#calculate and write to file
calc_wsj_sent(sent_type,new_words,wordDictCleaned,wsjCleaned)


In [ ]:
#calc s&p with wsj and OOB sent

#read in files
sAndP, wsjOOB, wsj = getFiles()
#join wsj and wsjOOB
grouped = joinWSJ(wsj,wsjOOB)
#calc average sent per day
groupedAvg = calcAvg(grouped)
#clean s&p
sAndPCleaned = clean(sAndP)
#join s&p with avg sent
sameDate = join_up(sAndPCleaned,groupedAvg)
#save file
saveFile(sameDate)

